In [66]:
import os
import geopandas as gpd

In [68]:
counties = gpd.read_file(os.path.abspath('data/Example_Counties.shp'))
roads = gpd.read_file(os.path.abspath('data/Example_Roads.shp'))

DriverError: Unable to open C:\Users\edgos\Documents\GitHub\egm722_assessment\data\Example_Counties.shx or C:\Users\edgos\Documents\GitHub\egm722_assessment\data\Example_Counties.SHX. Set SHAPE_RESTORE_SHX config option to YES to restore or create it.

In [24]:
counties.head()

,CTYUA23CD,CTYUA23NM,CTYUA23NMW,BNG_E,BNG_N,LONG,LAT,GlobalID,geometry
0,E06000044,Portsmouth,None,465619,101352,-1.07006,50.8080,71dabb10-dec3-411a-a75b-748c4715b8e8,"MULTIPOLYGON (((465574.900 94946.901, 465557.6..."
1,E06000045,Southampton,None,442303,113700,-1.39952,50.9212,b6f2a81e-69c0-4c14-bde0-7ccda131180d,"MULTIPOLYGON (((436895.800 114450.300, 436899...."
2,E06000046,Isle of Wight,None,447183,85949,-1.33366,50.6713,0a1b2ba6-e98f-4b3c-8c6a-63530e3a5dc9,"MULTIPOLYGON (((463996.499 93783.604, 463970.5..."
3,E06000054,Wiltshire,None,405209,158863,-1.92661,51.3288,599d3f46-6bda-4431-af0f-824ce4a9e594,"POLYGON ((413668.497 198850.399, 413752.300 19..."
4,E06000058,"Bournemouth, Christchurch and Poole",None,410815,94066,-1.84807,50.7461,9cedba33-8504-454e-a769-1b3d0b17f23f,"MULTIPOLYGON (((401164.904 90142.396, 401552.0..."


In [12]:
#This function ensures that the both the counties and the roads data is in the same CRS. Having it in BNG ensures the measurements are in metres
#If there is an issue with the data and there isn't a CRS it will return an error message
def ensure_bng(gdf):
    if gdf.crs is None:
        raise ValueError("GeoDataFrame has no coordinate reference system.")
    if gdf.crs.to_epsg() != 27700:
        gdf = gdf.to_crs(epsg=27700)
    return gdf

counties = ensure_bng(counties)
roads = ensure_bng(roads)

print("Counties CRS:", counties.crs)
print("Roads CRS:", roads.crs)

Counties CRS: EPSG:27700
Roads CRS: EPSG:27700


In [52]:
#Here a specific County is selected. 
county_name = "Wiltshire"
target_county = counties[counties["CTYUA23NM"] == county_name]

if target_county.empty:
    raise ValueError("No county of that name can be found in the data")

#Create a spatial join between roads and target county
roads_in_county = gpd.sjoin(roads, target_county, how="inner", predicate="intersects")

In [56]:
#The script is focussed specifically on classified roads
#Looking at the unique values of the "class" column those can be defined
roads_in_county["class"].unique()

array(['Unclassified', 'Classified Unnumbered', 'Unknown', 'A Road',
       'B Road', 'Not Classified'], dtype=object)

In [44]:
#
for index, row in roads_in_county.iterrows():
    roads_in_county.loc[index, 'Length'] = row['geometry'].length
    
roads_in_county.head()

,fictitious,identifier,class,roadNumber,name1,name1_lang,name2,name2_lang,formOfWay,length,...,Length,index_right,CTYUA23CD,CTYUA23NM,CTYUA23NMW,BNG_E,BNG_N,LONG,LAT,GlobalID
403,false,0277B441-59AD-467B-A2ED-06972052B6DF,Unclassified,None,New House Lane,None,None,None,Single Carriageway,707,...,677.805125,3,E06000054,Wiltshire,None,405209,158863,-1.92661,51.3288,599d3f46-6bda-4431-af0f-824ce4a9e594
2635,false,31326D0C-7624-4573-B9A2-DC0A5905B29E,Classified Unnumbered,None,None,None,None,None,Single Carriageway,1223,...,271.549180,3,E06000054,Wiltshire,None,405209,158863,-1.92661,51.3288,599d3f46-6bda-4431-af0f-824ce4a9e594
2735,false,34F9E0BB-5B06-4C78-ABCE-4B46CD6E891A,Unknown,None,None,None,None,None,Single Carriageway,2896,...,1249.750235,3,E06000054,Wiltshire,None,405209,158863,-1.92661,51.3288,599d3f46-6bda-4431-af0f-824ce4a9e594
5785,false,FAB8FFC0-ECAE-4FA7-BA4A-B748374E1908,Classified Unnumbered,None,Shoddesden Lane,None,None,None,Single Carriageway,836,...,158.824091,3,E06000054,Wiltshire,None,405209,158863,-1.92661,51.3288,599d3f46-6bda-4431-af0f-824ce4a9e594
9802,false,BAAACDFC-6DA0-4362-AA08-282AC4FBBAC4,Unclassified,None,Giles Lane,None,None,None,Single Carriageway,193,...,133.164294,3,E06000054,Wiltshire,None,405209,158863,-1.92661,51.3288,599d3f46-6bda-4431-af0f-824ce4a9e594


In [49]:
sum_total_roads = roads_in_county['Length'].sum() / 1000
sum_motorway = roads_in_county[roads_in_county['class'] == 'Motorway']['Length'].sum() / 1000
sum_a_road = roads_in_county[roads_in_county['class'] == 'A Road']['Length'].sum() / 1000
sum_b_road = roads_in_county[roads_in_county['class'] == 'B Road']['Length'].sum() / 1000
sum_class_unnum = roads_in_county[roads_in_county['class'] == 'Classified Unnumbered']['Length'].sum() / 1000
print(f'{sum_total_roads:.2f} total km of all roads')
print(f'{sum_motorway:.2f} total km of motorway')
print(f'{sum_a_road:.2f} total km of A roads')
print(f'{sum_b_road:.2f} total km of B roads')
print(f'{sum_class_unnum:.2f} total km of classified unnumbered roads')

12720.57 total km of all roads
172.92 total km of motorway
886.13 total km of A roads
582.81 total km of B roads
2030.83 total km of classified unnumbered roads
